# 1. Import libraries

In [ ]:
# import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import time
import pylab as pl
import pandas as pd
from numpy import loadtxt, zeros, ones, array, linspace, logspace
from pylab import scatter, show, title, xlabel, ylabel, plot, contour
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn import cross_validation, metrics
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# 2. Read .csv from provided dataset

In [2]:
csv_filename="data_mining.csv"
df=pd.read_csv(csv_filename, encoding='ISO-8859-1')
print(df.shape)
df.head()

(1459, 5)


,id,crawled,title,Original content,pageviews
0,9546141,2016-01-01T04:47:00+00:00,Kanye West Releases Awful New Song on New Year...,Is Drake inside Kanye West??s head? That??s ...,29478.0
1,9544391,2016-01-01T05:01:00+00:00,Why Everybody Should Work in Hollywood,I??m convinced that working in Hollywood is t...,24939.0
2,9544392,2016-01-01T05:01:00+00:00,Bill Cosby Tamir Rice And The Power of Prosecu...,What do Bill Cosby and Tamir Rice??s have in ...,13595.0
3,9544393,2016-01-01T05:01:00+00:00,Translating the ??Iliad??? Who Isn??t.,Pop quiz: Which is greater? (a) The number of ...,4752.0
4,9544394,2016-01-01T05:01:00+00:00,How Oprah Created a Profitable Weight-Loss Plan,Forget portion control: weight loss is a power...,26549.0


# 3. Data Preprocessing 
###  Replace Noisy Data, Remove Stopwords/Null values/Punctuations and Data stemming, lemmatize
### 3.1 Title Part:

In [3]:
#ps = PorterStemmer()
#sno = nltk.stem.SnowballStemmer('english')
lemma = WordNetLemmatizer()
df = df.dropna(how='any',axis=0)
title_doc = df[df.columns[2]].values
corpus =[]
for row in title_doc:
    #remove unwanted data
    title = re.sub ('[^a-zA-Z]', ' ', row)
    title = title.lower()   
    title = title.split()
    #remove stopwords
    stop = stopwords.words('english')
    #word stemming
    title = [lemma.lemmatize(item) for item in title if item not in stop]
    title = ' '.join(title)
    corpus.append(title)
print(corpus[:5])

['kanye west release awful new song new year eve', 'everybody work hollywood', 'bill cosby tamir rice power prosecutor', 'translating iliad', 'oprah created profitable weight loss plan']


### 3.2 Content Part:

In [4]:
content_doc = df[df.columns[3]].values
corpus2 =[]
for i in content_doc:
    content = re.sub ('[^a-zA-Z]', ' ', i)
    content = content.lower()
    content = content.split()
    stop = stopwords.words('english')
    ps = PorterStemmer()
    content = [lemma.lemmatize(item) for item in content if item not in stop]
    content = ' '.join(content)
    corpus2.append(content)
    
#print(corpus2[:5])

# 4. Document Term Matrix for Title
### 4.1 View first 4 rows

In [5]:
vec = CountVectorizer()
X_data_view = vec.fit_transform(corpus[:4])
data_matrix = pd.DataFrame(X_data_view.toarray(), columns=vec.get_feature_names())
print(data_matrix)

   awful  bill  cosby  eve  everybody  hollywood  iliad  kanye  new  power  \
0      1     0      0    1          0          0      0      1    2      0   
1      0     0      0    0          1          1      0      0    0      0   
2      0     1      1    0          0          0      0      0    0      1   
3      0     0      0    0          0          0      1      0    0      0   

   prosecutor  release  rice  song  tamir  translating  west  work  year  
0           0        1     0     1      0            0     1     0     1  
1           0        0     0     0      0            0     0     1     0  
2           1        0     1     0      1            0     0     0     0  
3           0        0     0     0      0            1     0     0     0  


### 4.2 Vector matrix for title

In [6]:
vec = CountVectorizer()
X = vec.fit_transform(corpus).toarray()
print(X)

# handle goal attrubte to binary 
# Catagorize the pageview count into popular and unpopular 
popular = df.pageviews >= 15000
unpopular = df.pageviews < 15000
df.loc[popular,'pageviews'] = 1
df.loc[unpopular,'pageviews'] = 0
y= df.iloc[:, 4].values
print(y)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[ 1.  1.  0. ...,  1.  0.  0.]


# 5. Document Term Matrix for Content

### 5.1 View first 3 rows for content

In [7]:
X_data_view2 = vec.fit_transform(corpus2[:3])
data_matrix2 = pd.DataFrame(X_data_view2.toarray(), columns=vec.get_feature_names())
print(data_matrix2)

   ability  able  absolutely  accompaniment  accountability  accountable  \
0        0     0           0              1               0            0   
1        1     2           1              0               1            0   
2        0     0           0              0               0            1   

   accusation  accused  act  actualized   ...     wrath  writer  xanax  year  \
0           0        0    0           0   ...         0       0      0     2   
1           0        0    1           1   ...         1       3      1     2   
2           1        2    0           0   ...         0       0      0     7   

   yeezy  yeezys  yes  yet  young  zealous  
0      8       1    0    1      0        0  
1      0       0    2    0      1        0  
2      0       0    0    0      1        1  

[3 rows x 920 columns]


### 5.2 Vector matrix for content

In [8]:
vec2 = CountVectorizer()
X1 = vec.fit_transform(corpus2).toarray()
print(X1)
y1= df.iloc[:, 4].values
print(y1)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[ 1.  1.  0. ...,  1.  0.  0.]


# 6. Split the Dataset into 70% training & 30% testing - Title

In [9]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=0)
#print(df.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1018, 3386)
(1018,)
(437, 3386)
(437,)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 7. Training Model for Title
#### To train the model, I divide the pageviews into two category- popular and unpopular (Binary Classification). I assume that if the page view is higher than 15000, then it is popular. Otherwise, it is unpopular.  
### 7.1 GaussianNB Model

In [11]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None)

### 7.2 Prediction and accuracy

In [12]:
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)
print("Accuracy", accuracy_score(y_test, y_pred, normalize = True))

Accuracy 0.613272311213


# 8. Training Model for Content

In [13]:
X1_train, X1_test, y1_train, y1_test = cross_validation.train_test_split(X1, y1, test_size=0.3, random_state=0)

### 8.1 GaussianNB Model for Content

In [14]:
clf = GaussianNB()
clf.fit(X1_train, y1_train)
y1_pred = clf.predict(X1_test)
confusion_matrix(y1_test, y1_pred)
print("Accuracy", accuracy_score(y1_test, y1_pred, normalize = True))

Accuracy 0.549199084668


# Tried Different Classification Models 
### Decision Tree Model for Title

In [15]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
clf_dt = dt.fit(X_train, y_train)
print ("Acurracy: ", clf_dt.score(X_test,y_test))

Acurracy:  0.572082379863


### Random Forest Model for Title

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
clf_rf = rf.fit(X_train,y_train)
print ("Acurracy: ", clf_rf.score(X_test,y_test))

Acurracy:  0.562929061785


### SVM Classifier for Title

In [17]:
from sklearn import svm
sv = svm.SVC()
clf_sv = sv.fit(X_train,y_train)
print ("Acurracy: ", clf_sv.score(X_test,y_test))

Acurracy:  0.569794050343


### Decision Tree Model for Content

In [18]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
clf_dt = dt.fit(X1_train, y1_train)

print ("Acurracy: ", clf_dt.score(X1_test,y1_test))

Acurracy:  0.62471395881


### Random Forest Model for content

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
clf_rf = rf.fit(X1_train,y1_train)
print ("Acurracy: ", clf_rf.score(X1_test,y1_test))

Acurracy:  0.617848970252


### SVM Classifier for content 

In [20]:
sv = svm.SVC()
clf_sv = sv.fit(X1_train,y1_train)
print ("Acurracy: ", clf_sv.score(X1_test,y1_test))

Acurracy:  0.606407322654
